# Semantic Kernel Process Framework ハンズオン（サンプル版）

Microsoft Semantic Kernel Process Frameworkのワークフローパターンを学習します。

## 学習内容
1. シーケンス 
2. ~~ファンアウト・ファンイン~~
3. ~~条件分岐~~
4. ~~繰り返し~~

## セットアップ

Process Frameworkを使用するために必要な依存関係をインストールします。

In [7]:
# 必要なライブラリのインポート
import asyncio
import time
from typing import Dict, Any, Optional
from datetime import datetime
from enum import Enum

# Semantic Kernel Process Framework
from semantic_kernel import Kernel
# from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from semantic_kernel.processes import ProcessBuilder
from semantic_kernel.processes.local_runtime.local_kernel_process import start as start_process
from semantic_kernel.functions import kernel_function
from semantic_kernel.processes import ProcessBuilder
from semantic_kernel.processes.local_runtime.local_kernel_process import start
from semantic_kernel.processes.local_runtime.local_event import KernelProcessEvent
from semantic_kernel.processes.kernel_process import KernelProcessStep, KernelProcessStepContext


# Kernel初期化
kernel = Kernel()
print("Kernel準備完了")

Kernel準備完了


## 1. シーケンス（Sequential Pattern）

最もシンプルなワークフローパターンです。処理が順次実行されます。

フロー: [Start] → [Step A] → [Step B] → [Step C] → [End]

In [14]:
class Events(Enum):
    Start = "Start"
    StartA = "StartA"
    StartB = "StartB"
    StepBCompleted = "StepBCompleted"

class StartStep(KernelProcessStep):
    @kernel_function()
    async def start(self, context: KernelProcessStepContext, user_input: str):
        print("--- START ---")
        await asyncio.sleep(2)
        print(f"User Input: {user_input}")
        await context.emit_event(process_event=Events.StartA, data=user_input)

class AStep(KernelProcessStep):
    @kernel_function()
    async def run_a(self, context: KernelProcessStepContext, data: str):
        print("--- STEP A start ---")
        print(f"Input: {data}")
        await asyncio.sleep(4)
        modified_data_a = f"{data} -> A"
        print(f"Output: {modified_data_a}")
        await context.emit_event(process_event=Events.StartB, data=modified_data_a)

class BStep(KernelProcessStep):
    @kernel_function()
    async def run_b(self, context: KernelProcessStepContext, data: str):
        print("--- STEP B start ---")
        print(f"Input: {data}")
        await asyncio.sleep(6)
        processed_data = f"{data} -> B"
        print(f"Output-B: {processed_data}")
        print("--- END ---")
        await context.emit_event(process_event=Events.StepBCompleted)


In [15]:
async def main(user_input: str):
    # Initialize the process builder
    builder = ProcessBuilder(name="SimpleProcess")

    # Add the steps to the process builder
    start_step = builder.add_step(step_type=StartStep)
    step_a = builder.add_step(step_type=AStep)
    step_b = builder.add_step(step_type=BStep)

    # Define the events and where to send them
    builder.on_input_event(Events.Start).send_event_to(target=start_step, function_name="start", parameter_name="user_input")
    start_step.on_event(Events.StartA).send_event_to(target=step_a, function_name="run_a", parameter_name="data")
    step_a.on_event(Events.StartB).send_event_to(target=step_b, function_name="run_b", parameter_name="data")
    step_b.on_event(Events.StepBCompleted).stop_process()

    # Build the process
    kernel_process = builder.build()

    # Start the process
    await start(
        process=kernel_process,
        kernel=Kernel(),
        initial_event=KernelProcessEvent(id=Events.Start, data=user_input),
    )

In [16]:
await main(user_input="Hello!")

--- START ---
User Input: Hello!
--- STEP A start ---
Input: Hello!
User Input: Hello!
--- STEP A start ---
Input: Hello!
Output: Hello! -> A
--- STEP B start ---
Input: Hello! -> A
Output: Hello! -> A
--- STEP B start ---
Input: Hello! -> A
Output-B: Hello! -> A -> B
--- END ---
Output-B: Hello! -> A -> B
--- END ---
